## Connecting Energy and population data

This was work was done mostly in Jupyter Labs to have a local connection so I can push data to MySql and create a new scheme that connects the data sets for ease of analysis and pulling information.

In [1]:
import pandas as pd
import requests

In [2]:
!pip install sqlalchemy
!pip install pymysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.6 MB/s eta 0:00:00


In [3]:
# Define the parameters to be passed into the url.
country = "DE"
city = "Berlin"
API_key = "Your API KEY"

# Reference the parameters in the url.
city_request = requests.get(f"https://api.api-ninjas.com/v1/city?name={city}&country={country}", headers={'X-Api-Key': API_key})
city_json = city_request.json()
city_json

[{'name': 'Berlin',
  'latitude': 52.5167,
  'longitude': 13.3833,
  'country': 'DE',
  'population': 3644826,
  'region': 'N/A',
  'is_capital': True}]

In [4]:
countries = ['Germany','Spain', 'Norway', 'Slovakia']
# base_url = ''
API_key = "YOUR API KEY"
historical_populations = []
#https://api.api-ninjas.com/v1/population?country=Germany

In [5]:
for country in countries:
    historical_populations.append(requests.get(f'https://api.api-ninjas.com/v1/population?country={country}', headers = {'X-Api-Key': API_key}).json())

In [6]:
populations = historical_populations


In [7]:
 pd.DataFrame(historical_populations)

,historical_population,population_forecast,country_name
0,"[{'year': 2024, 'population': 84552242, 'yearl...","[{'year': 2025, 'population': 84075075, 'yearl...",Germany
1,"[{'year': 2024, 'population': 47910526, 'yearl...","[{'year': 2025, 'population': 47889958, 'yearl...",Spain
2,"[{'year': 2024, 'population': 5576660, 'yearly...","[{'year': 2025, 'population': 5623071, 'yearly...",Norway
3,"[{'year': 2024, 'population': 5506760, 'yearly...","[{'year': 2025, 'population': 5474881, 'yearly...",Slovakia


In [10]:
populations_dict = {
    'country': [],
    'year': [],
    'population': [],
    'urban_pop_percent': []
}

for country in populations:
    country_name = country['country_name']

    for item in country['historical_population']:
        populations_dict['country'].append(country_name)
        populations_dict['year'].append(item['year'])
        populations_dict['population'].append(item['population'])
        populations_dict['urban_pop_percent'].append(item['urban_population_pct'])


country_populations = pd.DataFrame(populations_dict)
country_populations

country_populations.to_csv('country_populations.csv')
country_populations

## Below is the connecting of population and energy data

In [11]:
countries = {'country_name':['Germany','Spain', 'Norway', 'Slovak Republic']}
countries = pd.DataFrame(countries)
countries

,country_name
0,Germany
1,Spain
2,Norway
3,Slovak Republic


In [ ]:
country_populations = pd.read_csv('country_populations.csv')

In [ ]:
country_populations = pd.DataFrame.rename(country_populations, columns = {'country':'country_name', 'year':'year',
                                                                          'population':'population',
                                                                          'urban_pop_percent': 'urban_pop_percent'}).drop(axis=1,
                                                                                                                          columns ='Unnamed: 0')
country_populations


,country_name,year,population,urban_pop_percent
0,Germany,2024,84552242,76.0
1,Germany,2023,84548231,75.9
2,Germany,2022,84086227,76.3
3,Germany,2020,83628708,76.4
4,Germany,2015,82077550,76.9
...,...,...,...,...
63,Slovakia,1975,4739166,46.4
64,Slovakia,1970,4525871,41.2
65,Slovakia,1965,4365047,37.2
66,Slovakia,1960,4124442,33.6


In [ ]:
country_populations['country_name'] = country_populations['country_name'].replace(
    "Slovakia", "Slovak Republic"
)


In [ ]:
country_populations['country_name'] = (
    country_populations['country_name']
    .astype(str)
    .str.strip()          # remove invisible edges
    .str.replace('\u00A0', ' ',

                 regex=False)  # remove non-breaking space
)

In [ ]:
country_populations.head(50)


,country_name,year,population,urban_pop_percent
0,Germany,2024,84552242,76.0
1,Germany,2023,84548231,75.9
2,Germany,2022,84086227,76.3
3,Germany,2020,83628708,76.4
4,Germany,2015,82077550,76.9
5,Germany,2010,80830924,77.0
6,Germany,2005,82046731,75.6
7,Germany,2000,81797258,74.7
8,Germany,1995,82019890,73.2
9,Germany,1990,79707959,72.6


In [ ]:
slovak= country_populations['country_name'] == "Slovak Republic"
country_populations.loc[slovak]


,country_name,year,population,urban_pop_percent
51,Slovak Republic,2024,5506760,53.5
52,Slovak Republic,2023,5518055,53.3
53,Slovak Republic,2022,5473197,53.7
54,Slovak Republic,2020,5455024,53.7
55,Slovak Republic,2015,5420278,54.1
56,Slovak Republic,2010,5391851,54.8
57,Slovak Republic,2005,5376695,55.8
58,Slovak Republic,2000,5379999,56.4
59,Slovak Republic,1995,5354282,56.8
60,Slovak Republic,1990,5266078,56.7


In [12]:
schema = "EuroEnergy"
host = "127.0.0.1"
user = "root"
password = "Your PASSWORD"
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

## MYSQL Code

CREATE DATABASE EuroEnergy;

CREATE TABLE countries (
	country_id INT AUTO_INCREMENT,
    country_name VARCHAR(255) NOT NULL,
    PRIMARY KEY (country_id)
    );


CREATE TABLE historical_populations(
	population_id INT AUTO_INCREMENT,
	country_name VARCHAR(255) NOT NULL,
    year YEAR NOT NULL,
    population FLOAT NOT NULL,
    urban_pop_percent FLOAT NOT NULL,
    country_id INT,
    PRIMARY KEY (population_id),
    Foreign KEY (country_id) References countries(country_id)


--- Use DROP TABLE ; as needed to make corrections


In [ ]:
countries.to_sql('countries',
                  if_exists='append',
                  con=connection_string,
                  index=False)

4

In [ ]:
countries_from_sql = pd.read_sql("countries", con=connection_string)
countries_from_sql

,country_id,country_name
0,1,Germany
1,2,Spain
2,3,Norway
3,4,Slovak Republic


In [ ]:
country_populations = country_populations.merge(countries_from_sql, on ='country_name', how = 'right')
country_populations


,country_name,year,population,urban_pop_percent,country_id
0,Germany,2024,84552242,76.0,1
1,Germany,2023,84548231,75.9,1
2,Germany,2022,84086227,76.3,1
3,Germany,2020,83628708,76.4,1
4,Germany,2015,82077550,76.9,1
...,...,...,...,...,...
63,Slovak Republic,1975,4739166,46.4,4
64,Slovak Republic,1970,4525871,41.2,4
65,Slovak Republic,1965,4365047,37.2,4
66,Slovak Republic,1960,4124442,33.6,4


In [ ]:
country_populations.to_sql('historical_populations',
                if_exists='append',
                con=connection_string,
                index=False)

68

In [ ]:
historical_populations = pd.read_sql("historical_populations", con=connection_string)
historical_populations


,population_id,country_name,year,population,urban_pop_percent,country_id
0,1,Germany,2024,84552200.0,76.0,1
1,2,Germany,2023,84548200.0,75.9,1
2,3,Germany,2022,84086200.0,76.3,1
3,4,Germany,2020,83628700.0,76.4,1
4,5,Germany,2015,82077600.0,76.9,1
...,...,...,...,...,...,...
63,64,Slovak Republic,1975,4739170.0,46.4,4
64,65,Slovak Republic,1970,4525870.0,41.2,4
65,66,Slovak Republic,1965,4365050.0,37.2,4
66,67,Slovak Republic,1960,4124440.0,33.6,4


In [ ]:
historical_populations = pd.DataFrame.rename(historical_populations, columns = {'country_name':'country',
                                                                                'year':'year',
                                                                                'population':'population',
                                                                                'urban_pop_percent': 'urban_pop_percent'})

In [ ]:
historical_populations

,population_id,country,year,population,urban_pop_percent,country_id
0,1,Germany,2024,84552200.0,76.0,1
1,2,Germany,2023,84548200.0,75.9,1
2,3,Germany,2022,84086200.0,76.3,1
3,4,Germany,2020,83628700.0,76.4,1
4,5,Germany,2015,82077600.0,76.9,1
...,...,...,...,...,...,...
63,64,Slovak Republic,1975,4739170.0,46.4,4
64,65,Slovak Republic,1970,4525870.0,41.2,4
65,66,Slovak Republic,1965,4365050.0,37.2,4
66,67,Slovak Republic,1960,4124440.0,33.6,4


In [ ]:
MES_data = pd.read_csv('MES_data_1_clean (2).csv')

In [ ]:
MES_data

MES_data['date'] = pd.to_datetime(MES_data['date'], errors='coerce')
MES_data.info()

#countries = ["Germany", "Spain", "Norway", "Slovak Republic"]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154870 entries, 0 to 154869
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   country  154870 non-null  object        
 1   date     154870 non-null  datetime64[ns]
 2   balance  154870 non-null  object        
 3   product  154870 non-null  object        
 4   value    154870 non-null  float64       
 5   unit     154870 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 7.1+ MB


In [ ]:
countries = ["Slovak Republic", "Germany", "Spain", "Norway"]

filtered = MES_data[MES_data['country'].isin(countries)].copy()
filtered

,country,date,balance,product,value,unit
189,Germany,2025-08-01,Net Electricity Production,Electricity,36375.5605,GWh
190,Germany,2025-08-01,Net Electricity Production,Hydro,2242.6142,GWh
191,Germany,2025-08-01,Net Electricity Production,Total Combustible Fuels,15221.0789,GWh
192,Germany,2025-08-01,Net Electricity Production,"Coal, Peat and Manufactured Gases",6390.6006,GWh
193,Germany,2025-08-01,Net Electricity Production,Oil and Petroleum Products,413.5329,GWh
...,...,...,...,...,...,...
154673,Spain,2010-01-01,Total Exports,Electricity,1055.1970,GWh
154674,Spain,2010-01-01,Used for pumped storage,Electricity,566.3050,GWh
154675,Spain,2010-01-01,Distribution Losses,Electricity,2500.4510,GWh
154676,Spain,2010-01-01,Final Consumption (Calculated),Electricity,23046.8040,GWh


In [ ]:
filtered = MES_data[MES_data['country'].isin(countries)].copy()
filtered

filtered['year'] = filtered['date'].dt.year.copy()

filtered['year'].unique
filtered_2024 = filtered[filtered['year'] <= 2024].copy()
filtered_2024['country']  = filtered_2024['country'].str.strip()
filtered_2024['year'].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014,
       2013, 2012, 2011, 2010], dtype=int32)

In [ ]:
filtered_pop_2010 = historical_populations[historical_populations['year'] > 2009].copy()
filtered_pop_2010['country'] = filtered_pop_2010['country'].str.strip()
filtered_pop_2010['year'].unique()

filtered_pop_2010['population_recorded_in'] = filtered_pop_2010['year']
filtered_pop_2010

,population_id,country,year,population,urban_pop_percent,country_id,population_recorded_in
0,1,Germany,2024,84552200.0,76.0,1,2024
1,2,Germany,2023,84548200.0,75.9,1,2023
2,3,Germany,2022,84086200.0,76.3,1,2022
3,4,Germany,2020,83628700.0,76.4,1,2020
4,5,Germany,2015,82077600.0,76.9,1,2015
5,6,Germany,2010,80830900.0,77.0,1,2010
17,18,Spain,2024,47910500.0,79.1,2,2024
18,19,Spain,2023,47911600.0,79.0,2,2023
19,20,Spain,2022,47828400.0,78.9,2,2022
20,21,Spain,2020,47679500.0,78.7,2,2020


In [ ]:
filtered_2024['year'] = filtered_2024['year'].astype(int) # the years are int32 and int64 so we need them exactly as the same dtype
filtered_pop_2010['year'] = filtered_pop_2010['year'].astype(int)

energy_population_data = pd.merge_asof(filtered_2024.sort_values('year'), # this is a differnt type of merge because we need to back fill info
                                       filtered_pop_2010.sort_values('year'),# so that is fills na's that don't have a 1-1 year match.
    left_on='year',
    right_on='year',
    by='country',
    direction='backward' # this allows values in 2019 that don't have a year match in historical populations to be filled in with inform from the last census in 2015
)



energy_population_data = energy_population_data.drop(axis = 1, columns = ['unit','country_id'])

energy_population_data = pd.DataFrame.rename(energy_population_data,
                                           columns = {'country':'country','date':'date_of_energy_records',
                                                        'balance':'balance','product':'product',
                                                       'value': 'value(GWh)',
                                                       'year':'year',
                                                       'population_id':'population_id',
                                                       'population':'population',
                                                       'urban_pop_percent': 'Pecent_population_urban',
                                                       'country_id':'country_id'})



In [ ]:
energy_population_data

,country,date_of_energy_records,balance,product,value(GWh),year,population_id,population,Pecent_population_urban,population_recorded_in
0,Spain,2010-01-01,Net Electricity Production,"Total Renewables (Hydro, Geo, Solar, Wind, Other)",10143.8600,2010,23,46840500.0,78.4,2010
1,Spain,2010-01-01,Net Electricity Production,Not Specified,11.9170,2010,23,46840500.0,78.4,2010
2,Spain,2010-01-01,Net Electricity Production,Solar,273.6020,2010,23,46840500.0,78.4,2010
3,Spain,2010-01-01,Net Electricity Production,Other Combustible Non-Renewables,45.8970,2010,23,46840500.0,78.4,2010
4,Spain,2010-01-01,Net Electricity Production,Combustible Renewables,279.6320,2010,23,46840500.0,78.4,2010
...,...,...,...,...,...,...,...,...,...,...
13247,Norway,2024-12-01,Net Electricity Production,Electricity,15846.6010,2024,35,5576660.0,85.5,2024
13248,Germany,2024-12-01,Net Electricity Production,Hydro,2241.2938,2024,1,84552200.0,76.0,2024
13249,Germany,2024-12-01,Net Electricity Production,Electricity,42748.3323,2024,1,84552200.0,76.0,2024
13250,Germany,2024-12-01,Net Electricity Production,Wind,15408.7563,2024,1,84552200.0,76.0,2024


In [ ]:
energy_population_data

earliest_date = energy_population_data["date"].min()
latest_date = energy_population_data["date"].max()
print(f"Date range: {earliest_date} to {latest_date}")

Date range: 2010-01-01 00:00:00 to 2024-12-01 00:00:00


In [ ]:
#slovak_mask = energy_population_data['country_name'] == 'Slovak Republic'
energy_population_data.head()


,country,date,balance,product,value,unit,year,population_id,population,urban_pop_percent,country_id,population_recorded_in
0,Spain,2010-01-01,Net Electricity Production,"Total Renewables (Hydro, Geo, Solar, Wind, Other)",10143.860,GWh,2010,23,46840500.0,78.4,2,2010
1,Spain,2010-01-01,Net Electricity Production,Not Specified,11.917,GWh,2010,23,46840500.0,78.4,2,2010
2,Spain,2010-01-01,Net Electricity Production,Solar,273.602,GWh,2010,23,46840500.0,78.4,2,2010
3,Spain,2010-01-01,Net Electricity Production,Other Combustible Non-Renewables,45.897,GWh,2010,23,46840500.0,78.4,2,2010
4,Spain,2010-01-01,Net Electricity Production,Combustible Renewables,279.632,GWh,2010,23,46840500.0,78.4,2,2010


In [ ]:
energy_population_data.isna().sum()

country                   0
date                      0
balance                   0
product                   0
value                     0
unit                      0
year                      0
population_id             0
population                0
urban_pop_percent         0
country_id                0
population_recorded_in    0
dtype: int64

In [ ]:
energy_population_data.to_csv('energy_population_data.csv')
countries.to_csv('countries.csv')

historical_populations.to_csv('historical_populations')

In [ ]:
from IPython.display import FileLink

FileLink("energy_population_data.csv")

/Users/jessicahiggins/Desktop/Data Analytics/Final Project/Data_for_possible_projects/energy_population_data.csv

In [ ]:
energy_data = pd.read_csv('energy_population_data.csv')
energy_data = energy_data.drop(axis=1, columns = 'Unnamed: 0')
energy_data.isna().sum()

country                    0
date_of_energy_records     0
balance                    0
product                    0
value(GWh)                 0
year                       0
population_id              0
population                 0
Pecent_population_urban    0
population_recorded_in     0
dtype: int64

## Connecting to historical API weather data until 2020

In [ ]:
import requests
import pandas as pd


In [ ]:
API_key = "YOUR API KEY"

cities = [
    'Berlin', 'Dortmund', 'Kassel', 'Munich',
    'Madrid', 'Bilbao', 'Seville', 'Valencia',
    'Oslo', 'Trondheim', 'Harstad', 'Alta',
    'Bratislava', 'Banksa', 'Kosice', 'Zilina'
]

countries = [
    'DE', 'DE', 'DE', 'DE',
    'ES', 'ES', 'ES', 'ES',
    'NO', 'NO', 'NO', 'NO',
    'SK', 'SK', 'SK', 'SK'
]

url = "https://api.api-ninjas.com/v1/geocoding"

cities_list = []
lat_list = []
lon_list = []
country_code_list = []

for c, country in zip(cities, countries):
    response = requests.get(
        url,
        params={'city': c, 'country': country},
        headers={'X-Api-Key': API_key}
    )
    data = response.json()

    if data:
        item = data[0]   # first match

        cities_list.append(item['name'])
        lat_list.append(item['latitude'])
        lon_list.append(item['longitude'])
        country_code_list.append(item['country'])


print(cities_list)
print(lat_list)
print(lon_list)
print(country_code_list)

['Berlin', 'Dortmund', 'Kassel', 'Munich', 'Madrid', 'Bilbao', 'Seville', 'Valencia', 'Oslo', 'Trondheim', 'Harstad', 'Alta', 'Bratislava', 'Košice', 'Žilina']
[52.5170365, 51.5142273, 51.3154546, 48.1371079, 40.4167047, 43.2630018, 37.3886303, 39.4697065, 59.9133301, 63.4305658, 68.92973835000001, 70.04962755, 48.1435149, 48.7172272, 49.2234674]
[13.3888599, 7.4652789, 9.4924096, 11.5753822, -3.7035825, -2.9350039, -5.9953403, -0.3763353, 10.7389701, 10.3951929, 16.520913478350344, 23.08254009804839, 17.108279, 21.2496774, 18.7393139]
['DE', 'DE', 'DE', 'DE', 'ES', 'ES', 'ES', 'ES', 'NO', 'NO', 'NO', 'NO', 'SK', 'SK', 'SK']


In [ ]:


city_dict = {
    c: {"lat": lat, "lon": lon, "country": country}
    for c, lat, lon, country in zip(cities_list, lat_list, lon_list, country_code_list)
}



In [ ]:
city_geo = pd.DataFrame(city_dict)
city_geo = city_geo.transpose()
city_geo

,lat,lon,country
Berlin,52.517037,13.38886,DE
Dortmund,51.514227,7.465279,DE
Kassel,51.315455,9.49241,DE
Munich,48.137108,11.575382,DE
Madrid,40.416705,-3.703582,ES
Bilbao,43.263002,-2.935004,ES
Seville,37.38863,-5.99534,ES
Valencia,39.469707,-0.376335,ES
Oslo,59.91333,10.73897,NO
Trondheim,63.430566,10.395193,NO


In [ ]:
city_geo[['lat','lon']] = city_geo[['lat','lon']].astype(float)
#city_geo = city_geo.round({'lat': 2, 'lon': 2})

In [ ]:
# city_geo[city] = city_geo[]
city_geo = city_geo.reset_index()
# city_geo
#city_geo = city_geo.drop(columns = 'level_0').copy()
city_geo = pd.DataFrame.rename(city_geo, columns = {'index':'city','lat	':'lat', 'lon':'lon','country':'country_code'})
city_geo

,city,lat,lon,country_code
0,Berlin,52.517037,13.38886,DE
1,Dortmund,51.514227,7.465279,DE
2,Kassel,51.315455,9.49241,DE
3,Munich,48.137108,11.575382,DE
4,Madrid,40.416705,-3.703582,ES
5,Bilbao,43.263002,-2.935004,ES
6,Seville,37.38863,-5.99534,ES
7,Valencia,39.469707,-0.376335,ES
8,Oslo,59.91333,10.73897,NO
9,Trondheim,63.430566,10.395193,NO


In [ ]:
city_geo.to_csv('city_coordinates.csv')

In [ ]:
import pandas as pd
#!pip install python-dotenv
from dotenv import load_dotenv
import os
load_dotenv()  # Load .env into environment
import requests
import time
time.sleep(20)

In [ ]:
# import requests

# API_KEY = "J6VGTNFGLX4KCM883GQXT3AHC"

# test_url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/London?key={API_KEY}"

# response = requests.get(test_url)
# print(response.status_code)

# print(response.text)

In [ ]:
## This was done in the code before because here we realised we need to conduct the pull in Collab
base_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"
API_KEY = "YOUR API KEY"
url = f"{base_url}/52.51703,13.38886/2010-01-01/2010-01-02?key={API_KEY}"
print("Requesting URL:", url)

print("Status code:", response.status_code)
print("Response headers:", response.headers)
print("Response text:", response.text[:300])
print(response.headers.get("Content-Type"))
print(response.text[:500])# print first 300 chars of response


Requesting URL: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/52.51703,13.38886/2010-01-01/2010-01-02?key=J6VGTNFGLX4KCM883GQXT3AHC
Status code: 200
Response headers: {'Date': 'Sat, 06 Dec 2025 13:45:30 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Cache-Control': 'no-cache, no-transform', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET, POST, DELETE, PUT', 'Access-Control-Allow-Headers': 'X-Requested-With, Content-Type, X-Codingpedia', 'X-Powered-By': 'Visual Crossing Weather', 'Content-Encoding': 'gzip'}
Response text: {"queryCost":1,"latitude":51.5072,"longitude":-0.1275,"resolvedAddress":"London","address":"London","timezone":"Europe/London","tzoffset":0.0,"description":"Cooling down with a chance of rain tomorrow, Monday & Tuesday.","days":[{"datetime":"2025-12-06","datetimeEpoch":1764979200,"tempmax":54.6,"tem
application/json
{"queryCost":1,"latitude":51.5072,"

In [ ]:
#https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/[location]/[date1]/[date2]?key=YOUR_API_KEY
daily_climate_info = []
base_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"
API_KEY = "YOUR API KEY"

for index, row in city_geo.iterrows():
    lat = row['lat']
    lon = row['lon']
    name = row['city']

    url = f"{base_url}/{lat},{lon}/2010-01-01/2024-12-31"

    params = {
        'key': API_KEY,   # ✔ KEY GOES HERE
        'unitGroup': 'metric',
        "include": "daily,events",
        'elements': 'datetime,tempmax,tempmin,temp,humidity,precip,windspeed,cloudcover,conditions',
        'lang': 'en',
        'locationNames': name
    }

    response = requests.get(url, params=params)

    if response.status_code == 429:
        print("Rate limit exceeded. Waiting for 10 seconds...")
        time.sleep(10)  # wait longer to reset quota
        response = requests.get(url, params=params)

    if response.status_code != 200:
        print(f"Error {response.status_code} for {lat}, {lon}")
        print(response.text)
        continue

    data = response.json()

Error 401 for 52.5170365, 13.3888599
No account found with API key 'j6vgtnfglx4kcm883gqxt3ahcf'
Error 401 for 51.5142273, 7.4652789
No account found with API key 'j6vgtnfglx4kcm883gqxt3ahcf'
Error 401 for 51.3154546, 9.4924096
No account found with provided API key
Error 401 for 48.1371079, 11.5753822
No account found with provided API key
Error 401 for 40.4167047, -3.7035825
No account found with API key 'j6vgtnfglx4kcm883gqxt3ahcf'
Error 401 for 43.2630018, -2.9350039
No account found with provided API key
Error 401 for 37.3886303, -5.9953403
No account found with provided API key
Error 401 for 39.4697065, -0.3763353
No account found with provided API key
Error 401 for 59.9133301, 10.7389701
No account found with provided API key
Error 401 for 63.4305658, 10.3951929
No account found with provided API key
Error 401 for 68.92973835000001, 16.520913478350344
No account found with provided API key
Error 401 for 70.04962755, 23.08254009804839
No account found with provided API key
Error 4

In [ ]:
print(response.headers.get("Content-Type"))
print(response.text[:500])

In [ ]:
#https://api.weatherbit.io/v2.0/normals?lat=35.5&lon=-75.5&start_day=02-02&end_day=03-01&tp=daily&key=API_KEY
API_KEY = a67d85f57b4145a8a77580fa31ed532b

https://api.weatherbit.io/v2.0/normals?lat={latitude}&lon={longitude}&start_day={02-02}&end_day={03-01}&tp=daily&key=API_KEY

{'error': 'API throttled. Please try again in a minute.'}

In [ ]:
# handle 429
if response.status_code == 429: print("Rate limit hit — waiting 10 seconds...")
time.sleep(10) response = requests.get(base_url, params=params) # still bad?

if response.status_code != 200: print(f"Error for {latitude}, {longitude}: {response.status_code}")
print(response.text) continue daily_climate_info = response.json()


""


In [ ]:
# Getting cities Weather Information

base_url = 'https://api.weatherbit.io/v2.0/normals'
headers =

In [ ]:
city_geo

,city,lat,lon,country_code
0,Berlin,52.517037,13.388860,DE
1,Dortmund,51.514227,7.465279,DE
2,Kassel,51.315455,9.492410,DE
3,Munich,48.137108,11.575382,DE
4,Madrid,40.416705,-3.703582,ES
5,Bilbao,43.263002,-2.935004,ES
6,Seville,37.388630,-5.995340,ES
7,Valencia,39.469707,-0.376335,ES
8,Oslo,59.913330,10.738970,NO
9,Trondheim,63.430566,10.395193,NO


In [ ]:
print(city_geo.columns)
print(repr(API_KEY))

Index(['city', 'lat', 'lon', 'country_code'], dtype='object')
'J6VGTNFGLX4KCM883GQXT3AHC'


In [13]:
"YOUR API KEY".strip().upper()

'YOUR API KEY'

In [ ]:
import requests
import pandas as pd
import time

daily_climate_info = []
base_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"
API_KEY = "YOUR API KEY"


lat = row['lat']
lon = row['lon']
  # <- FIXED

    url = f"{base_url}/{lat},{lon}/2010-01-01/2024-12-31"

    params = {
        'key': API_KEY,
        'unitGroup': 'metric',
        "include": "daily,events",
        'elements': 'datetime,tempmax,tempmin,temp,humidity,precip,windspeed,cloudcover,conditions',
        'lang': 'en'
    }

    print("USING KEY:", params["key"])
    print("HTTP Status:", response.status_code, "for", lat, lon)

    response = requests.get(url, params=params)

    if response.status_code == 429:
        print("Rate limit exceeded. Waiting for 10 seconds...")
        time.sleep(10)
        response = requests.get(url, params=params)

    if response.status_code != 200:
        print(f"Error {response.status_code} for {lat}, {lon}, {name}")
        print(response.text)
        continue

    data = response.json()
    daily_climate_info.append(data)


USING KEY: J6VGTNFGLX4KCM883GQXT3AHC
HTTP Status: 400 for 52.5170365 13.3888599
USING KEY: J6VGTNFGLX4KCM883GQXT3AHC
HTTP Status: 200 for 51.5142273 7.4652789
USING KEY: J6VGTNFGLX4KCM883GQXT3AHC
HTTP Status: 200 for 51.3154546 9.4924096
USING KEY: J6VGTNFGLX4KCM883GQXT3AHC
HTTP Status: 200 for 48.1371079 11.5753822
USING KEY: J6VGTNFGLX4KCM883GQXT3AHC
HTTP Status: 200 for 40.4167047 -3.7035825
USING KEY: J6VGTNFGLX4KCM883GQXT3AHC
HTTP Status: 200 for 43.2630018 -2.9350039
USING KEY: J6VGTNFGLX4KCM883GQXT3AHC
HTTP Status: 200 for 37.3886303 -5.9953403
USING KEY: J6VGTNFGLX4KCM883GQXT3AHC
HTTP Status: 200 for 39.4697065 -0.3763353
USING KEY: J6VGTNFGLX4KCM883GQXT3AHC
HTTP Status: 200 for 59.9133301 10.7389701
USING KEY: J6VGTNFGLX4KCM883GQXT3AHC
HTTP Status: 200 for 63.4305658 10.3951929
USING KEY: J6VGTNFGLX4KCM883GQXT3AHC
HTTP Status: 200 for 68.92973835000001 16.520913478350344
USING KEY: J6VGTNFGLX4KCM883GQXT3AHC
HTTP Status: 200 for 70.04962755 23.08254009804839
USING KEY: J6VGTNFG

In [ ]:
print(data.keys())

dict_keys(['queryCost', 'latitude', 'longitude', 'resolvedAddress', 'address', 'timezone', 'tzoffset', 'days'])


In [ ]:
print(data['days'][0])

{'datetime': '2010-01-01', 'tempmax': 3.9, 'tempmin': 0.9, 'temp': 1.7, 'humidity': 95.9, 'precip': 1.442, 'windspeed': 8.1, 'cloudcover': 95.7, 'conditions': 'Snow, Rain, Overcast'}


In [ ]:
import json

print(json.dumps(data['days'][0], indent=2))


{
  "datetime": "2010-01-01",
  "tempmax": 3.9,
  "tempmin": 0.9,
  "temp": 1.7,
  "humidity": 95.9,
  "precip": 1.442,
  "windspeed": 8.1,
  "cloudcover": 95.7,
  "conditions": "Snow, Rain, Overcast"
}


In [ ]:
import pandas as pd

df = pd.DataFrame(data['days'])
print(df.head())

     datetime  tempmax  tempmin  temp  humidity  precip  windspeed  \
0  2010-01-01      3.9      0.9   1.7      95.9   1.442        8.1   
1  2010-01-02      1.8     -4.2  -1.0      86.0   1.137       13.0   
2  2010-01-03     -3.2     -9.2  -5.1      81.6   3.305       11.3   
3  2010-01-04     -5.1    -14.1 -10.0      84.4   0.005        7.8   
4  2010-01-05     -2.2     -7.1  -4.5      83.6   0.494        4.7   

   cloudcover              conditions  
0        95.7    Snow, Rain, Overcast  
1        99.1    Snow, Rain, Overcast  
2        91.3          Snow, Overcast  
3        70.3  Snow, Partially cloudy  
4        94.7          Snow, Overcast  


In [ ]:
daily_data = data.get('days', [])

df = pd.DataFrame(daily_data)

# Add latitude and longitude columns from the JSON root (same for all rows in this response)
df['lat'] = data.get('latitude')
df['lon'] = data.get('longitude')

# Optional: convert datetime column to datetime type
df['datetime'] = pd.to_datetime(df['datetime'])

print(df.head())

Weather_data = pd.DataFrame(df)

    datetime  tempmax  tempmin  temp  humidity  precip  windspeed  cloudcover  \
0 2010-01-01      3.9      0.9   1.7      95.9   1.442        8.1        95.7   
1 2010-01-02      1.8     -4.2  -1.0      86.0   1.137       13.0        99.1   
2 2010-01-03     -3.2     -9.2  -5.1      81.6   3.305       11.3        91.3   
3 2010-01-04     -5.1    -14.1 -10.0      84.4   0.005        7.8        70.3   
4 2010-01-05     -2.2     -7.1  -4.5      83.6   0.494        4.7        94.7   

               conditions        lat        lon  
0    Snow, Rain, Overcast  49.223467  18.739314  
1    Snow, Rain, Overcast  49.223467  18.739314  
2          Snow, Overcast  49.223467  18.739314  
3  Snow, Partially cloudy  49.223467  18.739314  
4          Snow, Overcast  49.223467  18.739314  


In [ ]:
Weather_data.info()


Weather_data.to_csv('weather_data.csv')


from IPython.display import FileLink

FileLink("weather_data.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5479 entries, 0 to 5478
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    5479 non-null   datetime64[ns]
 1   tempmax     5479 non-null   float64       
 2   tempmin     5479 non-null   float64       
 3   temp        5479 non-null   float64       
 4   humidity    5479 non-null   float64       
 5   precip      5479 non-null   float64       
 6   windspeed   5479 non-null   float64       
 7   cloudcover  5479 non-null   float64       
 8   conditions  5479 non-null   object        
 9   lat         5479 non-null   float64       
 10  lon         5479 non-null   float64       
dtypes: datetime64[ns](1), float64(9), object(1)
memory usage: 471.0+ KB


/Users/jessicahiggins/Desktop/Data Analytics/Final Project/Data_for_possible_projects/weather_data.csv

In [ ]:
city_geo

,city,lat,lon,country_code
0,Berlin,52.517037,13.38886,DE
1,Dortmund,51.514227,7.465279,DE
2,Kassel,51.315455,9.49241,DE
3,Munich,48.137108,11.575382,DE
4,Madrid,40.416705,-3.703582,ES
5,Bilbao,43.263002,-2.935004,ES
6,Seville,37.38863,-5.99534,ES
7,Valencia,39.469707,-0.376335,ES
8,Oslo,59.91333,10.73897,NO
9,Trondheim,63.430566,10.395193,NO


In [ ]:
daily_climate_info = []  # to collect DataFrames for all cities
API_KEY = "YOUR API KEY"


for index, row in city_geo.iterrows():
    lat = row['lat']
    lon = row['lon']
    city_name = row['city']

    url = f"{base_url}/{lat},{lon}/2010-01-01/2024-12-31"
    params = {
        'key': API_KEY,
        'unitGroup': 'metric',
        'include': 'days',
        'elements': 'datetime,tempmax,tempmin,temp,humidity,precip,windspeed,cloudcover,conditions',
        'lang': 'en',
    }

    response = requests.get(url, params=params)

    if response.status_code != 200:
        print(f"Error {response.status_code} for {city_name} ({lat}, {lon})")
        continue

    data = response.json()
    daily_data = data.get('days', [])

    # Create DataFrame from daily data
    df = pd.DataFrame(daily_data)

    # Add city info columns
    df['city'] = city_name
    df['lat'] = lat
    df['lon'] = lon

    # Convert datetime to pandas datetime type (optional)
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Collect this city's data
    daily_climate_info.append(df)

# After the loop, combine all city DataFrames into one
all_cities_df = pd.concat(daily_climate_info, ignore_index=True)

print(all_cities_df.head())

In [ ]:
all_cities_df.info()